In [ ]:
--USE ROLE SYSADMIN;
--CREATE OR REPLACE DATABASE LOAN_DEMO_DB;
--CREATE OR REPLACE SCHEMA RAW;

use schema snow_intel_bank_demo.snow_intel_bank_demo;

-- 1. Customers (100 records)
CREATE OR REPLACE TABLE RAW_CUSTOMERS AS
SELECT 
    SEQ4() AS CUST_ID,
    RANDOM() % 500 + 350 AS CREDIT_SCORE, -- Scores between 350 and 850
    CASE (ABS(RANDOM()) % 4) 
        WHEN 0 THEN 'New York' WHEN 1 THEN 'Miami' WHEN 2 THEN 'Chicago' ELSE 'Los Angeles' 
    END AS CITY,
    'customer_' || CUST_ID || '@example.com' AS EMAIL,
    CURRENT_DATE() - (ABS(RANDOM()) % 3650) AS MEMBER_SINCE
FROM TABLE(GENERATOR(ROWCOUNT => 100));

-- 2. Branches (5 branches)
CREATE OR REPLACE TABLE RAW_BRANCHES (BRANCH_ID INT, BRANCH_NAME STRING, REGION STRING);
INSERT INTO RAW_BRANCHES VALUES (1, 'Wall Street', 'North'), (2, 'Brickell', 'South'), (3, 'Santa Monica', 'West'), (4, 'Loop Central', 'Midwest'), (5, 'Silicon Valley', 'West');

-- 3. Officers (20 officers)
CREATE OR REPLACE TABLE RAW_OFFICERS AS
SELECT 
    SEQ4() AS OFFICER_ID,
    'Officer ' || OFFICER_ID AS OFFICER_NAME,
    (ABS(RANDOM()) % 5) + 1 AS BRANCH_ID
FROM TABLE(GENERATOR(ROWCOUNT => 20));

-- 4. Applications (120 loans)
CREATE OR REPLACE TABLE RAW_APPLICATIONS AS
SELECT 
    SEQ4() + 1000 AS APP_ID,
    ABS(RANDOM()) % 100 AS CUST_ID,
    CASE (ABS(RANDOM()) % 4) 
        WHEN 0 THEN 'Mortgage' WHEN 1 THEN 'Personal' WHEN 2 THEN 'Auto' ELSE 'Business' 
    END AS LOAN_TYPE,
    (ABS(RANDOM()) % 500000) + 5000 AS REQUESTED_AMOUNT,
    CURRENT_DATE() - (ABS(RANDOM()) % 60) AS APPLICATION_DATE
FROM TABLE(GENERATOR(ROWCOUNT => 120));

-- 5. Status History (Tracking the flow)
CREATE OR REPLACE TABLE RAW_STATUS_HISTORY AS
SELECT 
    SEQ4() AS STATUS_ID,
    a.APP_ID,
    CASE (ABS(RANDOM()) % 3)
        WHEN 0 THEN 'Requested' WHEN 1 THEN 'Under Review' ELSE 'Approved'
    END AS STATUS_NAME,
    a.APPLICATION_DATE + (ABS(RANDOM()) % 5) AS UPDATED_AT
FROM RAW_APPLICATIONS a;

-- 6. Comments (For NLP with SI)
CREATE OR REPLACE TABLE RAW_COMMENTS AS
SELECT 
    SEQ4() AS COMMENT_ID,
    a.APP_ID,
    (ABS(RANDOM()) % 20) AS OFFICER_ID,
    CASE (ABS(RANDOM()) % 5)
        WHEN 0 THEN 'The applicant has a high debt-to-income ratio due to a recent mortgage, but their job stability in the tech sector is a strong compensating factor. Recommend approval with a slightly higher interest rate.'
        WHEN 1 THEN 'Credit score is borderline. However, internal banking history shows no late payments in 5 years. I suggest requesting a co-signer or a larger down payment to mitigate risk.'
        WHEN 2 THEN 'Application flagged for manual review: the reported income does not match the tax returns provided. Need to clarify if there is secondary freelance income not yet verified.'
        WHEN 3 THEN 'Excellent candidate. High net worth individual with significant assets in our wealth management division. Strategic client for the branch, prioritize disbursement.'
        WHEN 4 THEN 'Rejecting this application. The borrower has two active personal loans and a history of overdrafts in the last 90 days. Risk profile exceeds our current appetite for auto loans.'
    END AS COMMENT_TEXT
FROM RAW_APPLICATIONS a;

In [ ]:
-- SILVER

-- Silver Customers: Deduplication and cleaning
CREATE OR REPLACE TABLE SILVER_CUSTOMERS AS
SELECT 
    CUST_ID,
    UPPER(CITY) AS CITY,
    CREDIT_SCORE,
    EMAIL,
    MEMBER_SINCE
FROM RAW_CUSTOMERS;

-- Silver Officers: Join with branches
CREATE OR REPLACE TABLE SILVER_OFFICERS AS
SELECT 
    o.OFFICER_ID,
    o.OFFICER_NAME,
    b.BRANCH_NAME,
    b.REGION
FROM RAW_OFFICERS o
JOIN RAW_BRANCHES b ON o.BRANCH_ID = b.BRANCH_ID;

-- Silver Applications: Format currency and dates
CREATE OR REPLACE TABLE SILVER_APPLICATIONS AS
SELECT 
    APP_ID,
    CUST_ID,
    LOAN_TYPE,
    CAST(REQUESTED_AMOUNT AS NUMBER(12,2)) AS REQUESTED_AMOUNT,
    APPLICATION_DATE
FROM RAW_APPLICATIONS;

-- Silver Comments: Clean text for analysis
CREATE OR REPLACE TABLE SILVER_COMMENTS AS
SELECT 
    COMMENT_ID,
    APP_ID,
    OFFICER_ID,
    TRIM(COMMENT_TEXT) AS COMMENT_TEXT
FROM RAW_COMMENTS;

In [ ]:
-- Dimension: Customers
CREATE OR REPLACE TABLE DIM_CUSTOMERS (
    CUST_ID INT COMMENT 'Unique identifier for the banking customer',
    CITY STRING COMMENT 'City where the customer resides',
    CREDIT_SCORE INT COMMENT 'Credit rating from 350 to 850. Higher is better',
    EMAIL STRING COMMENT 'Customer contact email',
    MEMBER_SINCE DATE COMMENT 'Date when the customer opened their first account'
) AS SELECT * FROM SILVER_CUSTOMERS;

-- Dimension: Officers (Slowly Changing or flat)
CREATE OR REPLACE TABLE DIM_OFFICERS (
    OFFICER_ID INT PRIMARY KEY COMMENT 'Unique identifier for the bank loan officer',
    OFFICER_NAME STRING COMMENT 'Full name of the loan officer responsible for the application',
    BRANCH_NAME STRING COMMENT 'The specific bank branch name where the officer is located',
    REGION STRING COMMENT 'Geographic territory (North, South, East, West) managed by the officer'
) 
COMMENT = 'Dimension table containing details about bank employees who manage loan applications'
AS SELECT * FROM SILVER_OFFICERS;

-- Fact Table: Loans
CREATE OR REPLACE TABLE FACT_LOANS (
    APP_ID INT COMMENT 'Unique loan application ID',
    CUST_ID INT COMMENT 'Foreign key to Dim_Customers',
    OFFICER_ID INT COMMENT 'Foreign key to Dim_Officers',
    LOAN_TYPE STRING COMMENT 'Category of loan: Mortgage, Auto, Personal, or Business',
    REQUESTED_AMOUNT NUMBER(12,2) COMMENT 'Total money requested by the applicant in USD',
    APPLICATION_DATE DATE COMMENT 'The date the application was submitted',
    CURRENT_STATUS STRING COMMENT 'Latest state of the loan: Requested, Under Review, Approved, or Rejected',
    DAYS_OPEN INT COMMENT 'Number of days elapsed since the application was created'
) AS
SELECT 
    a.APP_ID,
    a.CUST_ID,
    o.OFFICER_ID,
    a.LOAN_TYPE,
    a.REQUESTED_AMOUNT,
    a.APPLICATION_DATE,
    h.STATUS_NAME AS CURRENT_STATUS,
    DATEDIFF('day', a.APPLICATION_DATE, CURRENT_DATE()) AS DAYS_OPEN
FROM SILVER_APPLICATIONS a
LEFT JOIN (
    SELECT APP_ID, STATUS_NAME FROM RAW_STATUS_HISTORY
    QUALIFY ROW_NUMBER() OVER (PARTITION BY APP_ID ORDER BY UPDATED_AT DESC) = 1
) h ON a.APP_ID = h.APP_ID
LEFT JOIN SILVER_COMMENTS c ON a.APP_ID = c.APP_ID
LEFT JOIN SILVER_OFFICERS o ON c.OFFICER_ID = o.OFFICER_ID;

In [ ]:
-- Roles creation
CREATE OR REPLACE ROLE ROLE_RISK;
CREATE OR REPLACE ROLE ROLE_SALES;

-- Specific Table Access
-- Risk sees everything related to credit
GRANT SELECT ON TABLE FACT_LOANS TO ROLE ROLE_RISK;
GRANT SELECT ON TABLE DIM_CUSTOMERS TO ROLE ROLE_RISK;

-- Sales only sees loan performance and officers
GRANT SELECT ON TABLE FACT_LOANS TO ROLE ROLE_SALES;
GRANT SELECT ON TABLE DIM_OFFICERS TO ROLE ROLE_SALES;

GRANT ROLE ROLE_RISK TO ROLE ACCOUNTADMIN;
GRANT ROLE ROLE_SALES TO ROLE ACCOUNTADMIN;